# Mask R-CNN - Compare Weights

Compare and check which weights have changed

In [ ]:
import os
import sys
import random
import math
import re
import time
import numpy as np
import tensorflow as tf
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import keras

# Root directory of the project
ROOT_DIR = "C:\\Users\\Martin\\Documents\\LEGOFinder\\Keras\\lego_object_detection\\maskrcnn"

%load_ext autoreload
%autoreload 2

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn import utils
from mrcnn import visualize
from mrcnn.visualize import display_images
import mrcnn.model as modellib
from mrcnn.model import log

from samples.lego import lego

%matplotlib inline 

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")


In [ ]:
# Path to Lego trained weights
LEGO_1_WEIGHTS_PATH = "C:/Users/Martin/Google Drive/Colab/maskrcnn/snapshots/lego20200530T0450/mask_rcnn_lego_0020.h5"
LEGO_2_WEIGHTS_PATH = "C:/Users/Martin/Google Drive/Colab/maskrcnn/snapshots/lego20200530T0450/mask_rcnn_lego_0025.h5"

# load either "train", "val" or "eval"
# DATASET = "eval"


## Configurations

In [ ]:
config = lego.LegoConfig()
LEGO_DIR = os.path.join(ROOT_DIR, "datasets", "lego")

In [ ]:
# Override the training configurations with a few
# changes for inferencing.
class TrainingConfig(config.__class__):
    # Run detection on one image at a time
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    DETECTION_MIN_CONFIDENCE = 0.8
    USE_RPN_ROIS = True
    USE_RANDOM_RPN_ROIS = False

config = TrainingConfig()
config.display()

## Notebook Preferences

In [ ]:
# Device to load the neural network on.
# Useful if you're training a model on the same 
# machine, in which case use CPU and leave the
# GPU for training.
DEVICE = "/cpu:0"  # /cpu:0 or /gpu:0

# Inspect the model in training or inference modes
# values: 'inference' or 'training'
# TODO: code for 'training' test mode not ready yet
TEST_MODE = "inference"

In [ ]:
def get_ax(rows=1, cols=1, size=16):
    """Return a Matplotlib Axes array to be used in
    all visualizations in the notebook. Provide a
    central point to control graph sizes.
    
    Adjust the size attribute to control how big to render images
    """
    _, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
    return ax

## Load Dataset

In [ ]:
# Load validation dataset
#dataset = lego.LegoDataset()
#dataset.load_lego(LEGO_DIR, DATASET)

# Must call before using the dataset
#dataset.prepare()

#print("Images: {}\nClasses: {}".format(len(dataset.image_ids), dataset.class_names))

## Load Model

In [ ]:
# Create models
with tf.device(DEVICE):
    model1 = modellib.MaskRCNN(mode="training", model_dir=MODEL_DIR, config=config)
    model2 = modellib.MaskRCNN(mode="training", model_dir=MODEL_DIR, config=config)

In [ ]:
# Set path to lego weights file

# Load weights
print("Loading weights ", LEGO_1_WEIGHTS_PATH)
model1.load_weights(LEGO_1_WEIGHTS_PATH, by_name=True)
print("Loading weights ", LEGO_2_WEIGHTS_PATH)
model2.load_weights(LEGO_2_WEIGHTS_PATH, by_name=True)

## Select Layers to Analyse

In [ ]:
selected_layers = "heads"

layer_regex = {
        # all layers but the backbone
        "heads": r"(mrcnn\_.*)|(rpn\_.*)|(fpn\_.*)",
        # From a specific Resnet stage and up
        "3+": r"(res3.*)|(bn3.*)|(res4.*)|(bn4.*)|(res5.*)|(bn5.*)|(mrcnn\_.*)|(rpn\_.*)|(fpn\_.*)",
        "4+": r"(res4.*)|(bn4.*)|(res5.*)|(bn5.*)|(mrcnn\_.*)|(rpn\_.*)|(fpn\_.*)",
        "5+": r"(res5.*)|(bn5.*)|(mrcnn\_.*)|(rpn\_.*)|(fpn\_.*)",
        # All layers
        "all": ".*",
        "all_but_rpn": r"^(?!rpn\_).*",
        "rpn": r"(rpn\_.*)",
    }

if selected_layers in layer_regex.keys():
    selected_layers = layer_regex[selected_layers]

## Compare Layer Weights

In [ ]:
# compare weights layer by layer
keras_model1 = model1.keras_model
keras_model2 = model2.keras_model

layers1 = keras_model1.inner_model.layers if hasattr(keras_model1, "inner_model") else keras_model1.layers
layers2 = keras_model2.inner_model.layers if hasattr(keras_model2, "inner_model") else keras_model2.layers

def check_layer(layer1, layer2, intend=0):

    # check only layers that have been selected
    selected = bool(re.fullmatch(selected_layers, layer1.name))
    if not selected:
        return

    #log("{}{:20}   ({})".format(" " * intend, layer1.name, layer1.__class__.__name__))

    weights1 = layer1.get_weights()
    weights2 = layer2.get_weights()

    len_weights1 = len(weights1)
    len_weights2 = len(weights2)

    if len_weights1 != len_weights2:
        print("4 - length mismatch: " + layer1.name + ", " + layer2.name)

    # weights consist of further lists
    weight_idx = 0
    for weight1, weight2 in zip(weights1, weights2):

        e = np.allclose(weight1, weight2, 1e-06, 1e-06) 
        if not e:
            print("6 - weights mismatch: " + layer1.name + ", " + str(weight_idx))
        weight_idx += 1

for layer1, layer2 in zip(layers1, layers2):

    # Is the layer a model?
    if layer1.__class__.__name__ == 'Model':

        assert layer2.__class__.__name__ == 'Model', "2 - model mismatch."
        
        print("In model: ", layer1.name)

        model1_layers = layer1.layers
        model2_layers = layer2.layers
        
        len_model1 = len(model1_layers)
        len_model2 = len(model2_layers)
        
        if len_model1 != len_model2:
            print("3 - length mismatch.")
        
        for l1, l2 in zip(model1_layers, model2_layers):

            if not l1.weights:
                continue

            # Check layer. If layer is a container, update inner layer.
            if l1.__class__.__name__ == 'TimeDistributed':
                check_layer(l1.layer, l2.layer, intend=4)
            else:
                check_layer(l1, l2, intend=4)

        continue

    if not layer1.weights:
        continue

    # Check layer. If layer is a container, update inner layer.
    if layer1.__class__.__name__ == 'TimeDistributed':
        check_layer(layer1.layer, layer2.layer, intend=0)
    else:
        check_layer(layer1, layer2, intend=0)
